In [1]:
# pip install torch transformers
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")

if torch.cuda.is_available():
    # pip install bitsandbytes accelerate
    from transformers import BitsAndBytesConfig

    device = "cuda"
    quantization_config = BitsAndBytesConfig(load_in_8bit=True)
    model = AutoModelForCausalLM.from_pretrained(
        "google/gemma-2b-it",
        quantization_config=quantization_config
    )
else:
    device = "cpu"
    model = AutoModelForCausalLM.from_pretrained(
        "google/gemma-2b-it",
        torch_dtype=torch.bfloat16
    )

`low_cpu_mem_usage` was None, now set to True since model is quantized.
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
input_text = """
Q:An 82-year-old male with congestive heart failure experiences rapid decompensation of his condition, manifesting as worsening dyspnea, edema, and increased fatigue. Labs reveal an increase in his serum creatinine from baseline. As part of the management of this acute change, the patient is given IV dobutamine to alleviate his symptoms. Which of the following effects occur as a result of this therapy?? {'A': 'Slowed atrioventricular conduction velocities', 'B': 'Increased myocardial oxygen consumption', 'C': 'Decreased heart rate', 'D': 'Increased systemic vascular resistance due to systemic vasoconstriction', 'E': 'Decreased cardiac contractility'},

Provide your answer as a JSON dictionary with the "option" and answer "text".
""".strip()

input_ids = tokenizer(input_text, return_tensors="pt").to(device)
outputs = model.generate(
    **input_ids,
    do_sample=True,
    max_new_tokens=512,
    temperature=1e-3,
)
print(tokenizer.decode(outputs[0]))

<bos>Q:An 82-year-old male with congestive heart failure experiences rapid decompensation of his condition, manifesting as worsening dyspnea, edema, and increased fatigue. Labs reveal an increase in his serum creatinine from baseline. As part of the management of this acute change, the patient is given IV dobutamine to alleviate his symptoms. Which of the following effects occur as a result of this therapy?? {'A': 'Slowed atrioventricular conduction velocities', 'B': 'Increased myocardial oxygen consumption', 'C': 'Decreased heart rate', 'D': 'Increased systemic vascular resistance due to systemic vasoconstriction', 'E': 'Decreased cardiac contractility'},

Provide your answer as a JSON dictionary with the "option" and answer "text".

```json
{
  "option": "B",
  "answer": "Increased myocardial oxygen consumption"
}
```<eos>
